# New Direct Inversion in Iterative Space (DIIS)

This method has been extensively used to solve self-consistent field (SCF) problems in the fields of quantum chemistry and physics. In this tutorial, we employ this method to accelerate the solving of fixed-point problems.

It should be noted that one potential issue with this method is that non-negative parameters cannot be guaranteed during optimization in the conventional DIIS approach. Although this issue can be addressed by explicitly introducing constraints to the linear combination coefficients, key concepts in DIIS, numerical issues may still arise, such as singular matrices or convergence problems.

In [ ]:
import logging

import numpy as np
from setup import prepare_argument_dict, prepare_grid_and_dens, print_results

from horton_part import (
    BasisFuncHelper,
    LinearISAWPart,
    check_pro_atom_parameters,
    compute_quantities,
    diis_solver_dyn,
    diis_solver_with_extra_constr,
    diis_spsolver,
)

logging.basicConfig(level=logging.ERROR, format="%(levelname)s:    %(message)s")
logger = logging.getLogger(__name__)

One can also use other DIIS methods and availiable methods are `diis_solver_spsolver` (default), `diis_solver_dyn`, `diis_solver_with_extra_constr`.

In [ ]:
def opt_propars_diis(
    bs_funcs,
    rho,
    propars,
    points,
    weights,
    threshold,
    density_cutoff,
    diis_size=8,
    diis_method=None,
):
    r"""
    Optimize parameters for proatom density functions using direct inversion in an iterative space (DIIS).

    Parameters
    ----------
    bs_funcs : 2D np.ndarray
        Basis functions array with shape (M, N), where 'M' is the number of basis functions
        and 'N' is the number of grid points.
    rho : 1D np.ndarray
        Spherically-averaged atomic density as a function of radial distance, with shape (N,).
    propars : 1D np.ndarray
        Pro-atom parameters with shape (M). 'M' is the number of basis functions.
    points : 1D np.ndarray
        Radial coordinates of grid points, with shape (N,).
    weights : 1D np.ndarray
        Weights for integration, including the angular part (4πr²), with shape (N,).
    threshold : float
        Convergence threshold for the iterative process.
    density_cutoff : float
        Density values below this cutoff are considered invalid.
    diis_size : int
        The size of DIIS subspace.
    diis_method : callable or None
        DIIS solver.

    Returns
    -------
    1D np.ndarray
        Optimized proatom parameters.

    Raises
    ------
    RuntimeError
        If the inner iteration does not converge.

    """
    diis_method = diis_method or diis_spsolver

    history_r = []
    history_x = []
    history_g = []

    logger.debug("            Iter.    dRMS      ")
    logger.debug("            -----    ------    ")

    def func_g(x):
        """The objective fixed-point equation."""
        pro_shells, _, _, ratio, _ = compute_quantities(
            rho, x, bs_funcs, density_cutoff
        )
        return np.einsum("ip,p->i", pro_shells * ratio, weights)

    version = "P"

    for irep in range(1000):
        g = func_g(propars)
        r = g - propars

        drms = np.linalg.norm(r)
        logger.debug(f"           {irep:<4}    {drms:.6E}")

        if drms < threshold:
            check_pro_atom_parameters(propars)
            return propars

        # Append trail & residual vectors to lists
        history_r.append(r)
        space_size = min(irep + 1, diis_size)
        r_list = history_r[-space_size:]

        if version == "P":
            # Anderson-Pulay version P
            history_x.append(propars.copy())
            x_list = history_x[-space_size:]
            x_tilde = diis_method(x_list, r_list, g)
            propars = func_g(x_tilde)
        else:
            # Anderson-Pulay version A
            history_g.append(g)
            g_list = history_g[-space_size:]
            propars = diis_method(g_list, r_list, g)

    raise RuntimeError("Error: inner iteration is not converge!")


def run_diis(diis_method):
    """Self-consistent solver."""
    mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = opt_propars_diis
    print("*" * 80)
    print(f"Results with {diis_method.__name__}".center(80, " "))
    print("*" * 80)
    kwargs["solver_options"] = {"diis_size": 8, "diis_method": diis_method}
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    print()

In [ ]:
run_diis(diis_spsolver)

In [ ]:
from cdiis import cdiis_algo

In [ ]:
def run_cdiis(**solver_options):
    """Self-consistent solver."""
    mol, grid, rho = prepare_grid_and_dens("data/h2o.fchk")
    kwargs = prepare_argument_dict(mol, grid, rho)
    kwargs["solver"] = cdiis_algo
    kwargs["solver_options"] = solver_options
    part = LinearISAWPart(**kwargs)
    part.do_all()
    print_results(part)
    print()

In [ ]:
run_cdiis(mode="R-CDIIS", param=1e-2)

In [ ]:
run_cdiis(mode="FD-CDIIS", sizediis=5)

In [ ]:
run_cdiis(mode="AD-CDIIS", param=1e-4)

In [ ]:
run_cdiis(mode="Roothaan")